In [1]:
import yfinance as yf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [2]:
# # Get all the S&P500 tickers
# path = './Data/S&P500/'
# dir_path = [name[:-4] for name in os.listdir(path)]
# print(dir_path)

In [3]:
# Get the history price MAX
GetInfo = yf.Ticker("AAPL")

data = GetInfo.history(period="max").reset_index()

data

/Users/sabrinachiang/miniforge3/envs/StockPrice/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-12-12 00:00:00-05:00,0.099319,0.099750,0.099319,0.099319,469033600,0.0,0.0
1,1980-12-15 00:00:00-05:00,0.094569,0.094569,0.094137,0.094137,175884800,0.0,0.0
2,1980-12-16 00:00:00-05:00,0.087659,0.087659,0.087228,0.087228,105728000,0.0,0.0
3,1980-12-17 00:00:00-05:00,0.089387,0.089818,0.089387,0.089387,86441600,0.0,0.0
4,1980-12-18 00:00:00-05:00,0.091978,0.092410,0.091978,0.091978,73449600,0.0,0.0
...,...,...,...,...,...,...,...,...
10873,2024-01-31 00:00:00-05:00,187.039993,187.100006,184.350006,184.399994,55467800,0.0,0.0
10874,2024-02-01 00:00:00-05:00,183.990005,186.949997,183.820007,186.860001,64885400,0.0,0.0
10875,2024-02-02 00:00:00-05:00,179.860001,187.330002,179.250000,185.850006,102518000,0.0,0.0
10876,2024-02-05 00:00:00-05:00,188.149994,189.250000,185.839996,187.679993,69668800,0.0,0.0


In [4]:
data = data[['Date', 'Close']]
data

,Date,Close
0,1980-12-12 00:00:00-05:00,0.099319
1,1980-12-15 00:00:00-05:00,0.094137
2,1980-12-16 00:00:00-05:00,0.087228
3,1980-12-17 00:00:00-05:00,0.089387
4,1980-12-18 00:00:00-05:00,0.091978
...,...,...
10873,2024-01-31 00:00:00-05:00,184.399994
10874,2024-02-01 00:00:00-05:00,186.860001
10875,2024-02-02 00:00:00-05:00,185.850006
10876,2024-02-05 00:00:00-05:00,187.679993


In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [6]:
# Generate the 
from copy import deepcopy as dc

def prepare_data(df, nsteps):
    '''
    for each day, we append n previous days 
    '''
    df = dc(df)
    df.set_index('Date', inplace = True)
    
    for i in range(1, nsteps + 1):
        df[f'Close(t-{i})D'] = df['Close'].shift(i)
        
    df.dropna(inplace= True)
    
    return df

In [7]:
lookback = 7
shift_df = prepare_data(data, lookback)

shift_df

,Close,Close(t-1)D,Close(t-2)D,Close(t-3)D,Close(t-4)D,Close(t-5)D,Close(t-6)D,Close(t-7)D
Date,,,,,,,,
1980-12-23 00:00:00-05:00,0.106660,0.102341,0.097592,0.091978,0.089387,0.087228,0.094137,0.099319
1980-12-24 00:00:00-05:00,0.112273,0.106660,0.102341,0.097592,0.091978,0.089387,0.087228,0.094137
1980-12-26 00:00:00-05:00,0.122637,0.112273,0.106660,0.102341,0.097592,0.091978,0.089387,0.087228
1980-12-29 00:00:00-05:00,0.124364,0.122637,0.112273,0.106660,0.102341,0.097592,0.091978,0.089387
1980-12-30 00:00:00-05:00,0.121342,0.124364,0.122637,0.112273,0.106660,0.102341,0.097592,0.091978
...,...,...,...,...,...,...,...,...
2024-01-31 00:00:00-05:00,184.399994,188.039993,191.729996,192.419998,194.169998,194.500000,195.179993,193.889999
2024-02-01 00:00:00-05:00,186.860001,184.399994,188.039993,191.729996,192.419998,194.169998,194.500000,195.179993
2024-02-02 00:00:00-05:00,185.850006,186.860001,184.399994,188.039993,191.729996,192.419998,194.169998,194.500000


In [8]:
shift_numpy = shift_df.to_numpy()
shift_numpy

array([[1.06659964e-01, 1.02341227e-01, 9.75915194e-02, ...,
        8.72276649e-02, 9.41371918e-02, 9.93186757e-02],
       [1.12273306e-01, 1.06659964e-01, 1.02341227e-01, ...,
        8.93866420e-02, 8.72276649e-02, 9.41371918e-02],
       [1.22637063e-01, 1.12273306e-01, 1.06659964e-01, ...,
        9.19781700e-02, 8.93866420e-02, 8.72276649e-02],
       ...,
       [1.85850006e+02, 1.86860001e+02, 1.84399994e+02, ...,
        1.92419998e+02, 1.94169998e+02, 1.94500000e+02],
       [1.87679993e+02, 1.85850006e+02, 1.86860001e+02, ...,
        1.91729996e+02, 1.92419998e+02, 1.94169998e+02],
       [1.89300003e+02, 1.87679993e+02, 1.85850006e+02, ...,
        1.88039993e+02, 1.91729996e+02, 1.92419998e+02]])

In [9]:
# Need to scale our data first
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))
shift_numpy = scaler.fit_transform(shift_numpy)

shift_numpy

array([[-0.99930672, -0.99935033, -0.99939829, ..., -0.99950293,
        -0.99943317, -0.99938085],
       [-0.99925004, -0.99930672, -0.99935033, ..., -0.99948113,
        -0.99950293, -0.99943317],
       [-0.99914539, -0.99925004, -0.99930672, ..., -0.99945497,
        -0.99948113, -0.99950293],
       ...,
       [ 0.87620669,  0.88640494,  0.86156542, ...,  0.94254612,
         0.96021646,  0.9635486 ],
       [ 0.89468468,  0.87620669,  0.88640494, ...,  0.93557893,
         0.94254612,  0.96021646],
       [ 0.91104247,  0.89468468,  0.87620669, ...,  0.89831973,
         0.93557893,  0.94254612]])

In [10]:
x = shift_numpy[:, 1:]
y = shift_numpy[:, 0]

x.shape, y.shape

((10871, 7), (10871,))

In [11]:
# reverses the x --> the array will be D-7, D-6, D-5, D-4, D-3, D-2, D-1
x = dc(np.flip(x, axis=1))
x

array([[-0.99938085, -0.99943317, -0.99950293, ..., -0.99945497,
        -0.99939829, -0.99935033],
       [-0.99943317, -0.99950293, -0.99948113, ..., -0.99939829,
        -0.99935033, -0.99930672],
       [-0.99950293, -0.99948113, -0.99945497, ..., -0.99935033,
        -0.99930672, -0.99925004],
       ...,
       [ 0.9635486 ,  0.96021646,  0.94254612, ...,  0.89831973,
         0.86156542,  0.88640494],
       [ 0.96021646,  0.94254612,  0.93557893, ...,  0.86156542,
         0.88640494,  0.87620669],
       [ 0.94254612,  0.93557893,  0.89831973, ...,  0.88640494,
         0.87620669,  0.89468468]])

In [12]:
# Split into train 90% and test 10%
 
split_index = int(len(x)*0.90)

split_index

9783

In [13]:
x_train = x[: split_index]
x_test = x[split_index:]

y_train = y[: split_index]
y_test = y[split_index:]

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((9783, 7), (1088, 7), (9783,), (1088,))

In [14]:
# Add another dimension for torch LSTM

x_train = x_train.reshape(x_train.shape[0], -1, 1)
x_test = x_test.reshape(x_test.shape[0], -1, 1)

y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((9783, 7, 1), (1088, 7, 1), (9783, 1), (1088, 1))

In [15]:
# convert to tensor

x_train = torch.tensor(x_train).float()
y_train = torch.tensor(y_train).float()

x_test = torch.tensor(x_test).float()
y_test = torch.tensor(y_test).float()

x_train.shape, x_test.shape, y_train.shape, y_test.shape

(torch.Size([9783, 7, 1]),
 torch.Size([1088, 7, 1]),
 torch.Size([9783, 1]),
 torch.Size([1088, 1]))

In [18]:
# create torch dataset object

from torch.utils.data import Dataset

class TimeSeriesData(Dataset):
    
    def __init__(self, x, y) -> None:
        self.x = x
        self.y = y
        
    def len(self):
        return len(self.x)
    
    def getitem(self, i) -> tuple:
        return self.x[i], self.y[i]

In [19]:
train_dataset = TimeSeriesData(x_train, y_train)
test_dataset = TimeSeriesData(x_test, y_test)

In [ ]:
# Load data with DataLoader

from torch.utils.data import DataLoader

batchsize = 15
train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle= True)
test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=True)